**660710689 กนกวรรณ นพพันธ์**

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import re

In [3]:
df =pd.read_csv('student.csv')

In [4]:
df

,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน
0,64001,สมชาย,20.0,ชาย,วิทยาศาสตร์,3.21,170.0,65.0,15.0,2024-02-13
1,64002,สมหญิง,21.0,หญิง,วิทย์,3.45,165.0,58.0,20.0,2023-12-31
2,64003,NaN,NaN,Male,Science,3.67,175.0,NaN,25.0,2023-12-30
3,64004,สมศักดิ์,22.0,ช,คณะวิทยาศาสตร์,-1.00,168.0,70.0,10.0,2024-01-31
4,64005,สมใจ,19.0,ญ,วิศวะ,3.89,160.0,52.0,30.0,2024-02-04
5,64006,กานต์,150.0,ชาย,Engineering,3.12,172.0,75.0,12.0,2024-01-28
6,64007,กิตติ,21.0,หญิง,วิศวกรรม,4.50,158.0,48.0,35.0,2024-02-08
7,64008,จิรา,20.0,F,ศิลปศาสตร์,2.98,180.0,82.0,18.0,2024-02-05
8,64009,ชนาธิป,23.0,M,Arts,3.34,163.0,55.0,22.0,2024-02-10
9,64010,ดารา,21.0,หญิง,ศิลป์,3.56,167.0,60.0,28.0,2023-12-18


ขั้นตอนที่ 1: สำรวจและแก้ไขค่าที่ผิดพลาด (Invalid Values)

In [5]:
df['GPA'] = df['GPA'].apply(lambda x: x if 0 <= x <= 4 else np.nan)

In [6]:
df['อายุ'] = df['อายุ'].apply(lambda x: x if x < 100 else np.nan)

In [7]:
print(df[['อายุ', 'GPA']].head(10))

   อายุ   GPA
0  20.0  3.21
1  21.0  3.45
2   NaN  3.67
3  22.0   NaN
4  19.0  3.89
5   NaN  3.12
6  21.0   NaN
7  20.0  2.98
8  23.0  3.34
9  21.0  3.56


ก่อนที่เราจะเติมค่าเฉลี่ย เราต้องแน่ใจก่อนว่าข้อมูลที่มีอยู่นั้นถูกต้อง (เกรดควรอยู่ระหว่าง 0-4) และใน อายุ มี 150 ซึ่งเป็นไปไม่ได้ เราจะเปลี่ยนค่าเหล่านี้ให้เป็นค่าว่าง (NaN) ก่อน

ขั้นตอนที่ 2: จัดการข้อมูลประเภทข้อความให้เป็นมาตรฐาน (Standardize Categorical Data)

In [8]:
gender_map = {'ชาย': 'ชาย', 'ช': 'ชาย', 'M': 'ชาย', 'Male': 'ชาย',
              'หญิง': 'หญิง', 'ญ': 'หญิง', 'F': 'หญิง', 'Female': 'หญิง'}
faculty_map = {'วิทยาศาสตร์': 'วิทยาศาสตร์', 'วิทย์': 'วิทยาศาสตร์', 'Science': 'วิทยาศาสตร์', 'คณะวิทยาศาสตร์': 'วิทยาศาสตร์', 'Sci': 'วิทยาศาสตร์',
               'วิศวะ': 'วิศวกรรมศาสตร์', 'Engineering': 'วิศวกรรมศาสตร์', 'วิศวกรรม': 'วิศวกรรมศาสตร์', 'วิศวฯ': 'วิศวกรรมศาสตร์', 'วิศวกรรมศาสตร์': 'วิศวกรรมศาสตร์',
               'ศิลปศาสตร์': 'ศิลปศาสตร์', 'Arts': 'ศิลปศาสตร์', 'ศิลป์': 'ศิลปศาสตร์', 'Liberal Arts': 'ศิลปศาสตร์'}

In [9]:
df['เพศ'] = df['เพศ'].map(gender_map)
df['คณะ'] = df['คณะ'].map(faculty_map)

In [10]:
print(df[['เพศ', 'คณะ']].head(15))

     เพศ             คณะ
0    ชาย     วิทยาศาสตร์
1   หญิง     วิทยาศาสตร์
2    ชาย     วิทยาศาสตร์
3    ชาย     วิทยาศาสตร์
4   หญิง  วิศวกรรมศาสตร์
5    ชาย  วิศวกรรมศาสตร์
6   หญิง  วิศวกรรมศาสตร์
7   หญิง      ศิลปศาสตร์
8    ชาย      ศิลปศาสตร์
9   หญิง      ศิลปศาสตร์
10   NaN             NaN
11   ชาย     วิทยาศาสตร์
12  หญิง  วิศวกรรมศาสตร์
13   ชาย      ศิลปศาสตร์
14  หญิง     วิทยาศาสตร์


ในข้อมูลดังกล่าวตรงคอลัมน์ 'เพศ' และ 'คณะ' มีการใส่ข้อมูลหลายรูปแบบ เช่น 'ชาย', 'ช', 'M' ซึ่งจริงๆ แล้วคือความหมายเดียวกัน เราต้องทำให้มันเป็นมาตรฐานเดียวกันก่อน เพื่อให้การวิเคราะห์หรือเติมค่าที่หายไปทำได้ถูกต้อง

ขั้นตอนที่ 3: จัดการข้อมูลที่หายไป (Handle Missing Values)

In [12]:
df.dropna(subset=['ชื่อ'], inplace=True)

print(f"จำนวนข้อมูลที่เหลือ: {len(df)} แถว")

จำนวนข้อมูลที่เหลือ: 19 แถว


ชื่อ: เป็นข้อมูลเฉพาะบุคคล ไม่สามารถเดาได้ และแถวที่มีชื่อหายไป (index=2) ก็มีข้อมูลอื่นหายไปด้วย การลบแถวนี้อาจเป็นทางเลือกที่ดี

In [13]:
numeric_cols = ['อายุ', 'GPA', 'ส่วนสูง_cm', 'น้ำหนัก_kg', 'ชั่วโมงเรียน_ต่อสัปดาห์']
for col in numeric_cols:
    mean_val = df[col].mean()
    df[col].fillna(mean_val, inplace=True)
    print(f"ค่าว่าง '{col}' ค่าเฉลี่ย: {mean_val:.2f}")

ค่าว่าง 'อายุ' ค่าเฉลี่ย: 20.88
ค่าว่าง 'GPA' ค่าเฉลี่ย: 3.42
ค่าว่าง 'ส่วนสูง_cm' ค่าเฉลี่ย: 167.28
ค่าว่าง 'น้ำหนัก_kg' ค่าเฉลี่ย: 63.61
ค่าว่าง 'ชั่วโมงเรียน_ต่อสัปดาห์' ค่าเฉลี่ย: 21.18


ข้อมูลตัวเลข: อายุ, GPA, ส่วนสูง, น้ำหนัก, ชั่วโมงเรียน เราจะใช้วิธี เติมด้วยค่าเฉลี่ย (Mean) ซึ่งเป็นวิธีที่นิยม

In [14]:
categorical_cols = ['เพศ', 'คณะ']
for col in categorical_cols:
    mode_val = df[col].mode()[0]
    df[col].fillna(mode_val, inplace=True)
    print(f"ค่าว่าง '{col}' ค่าที่พบบ่อย: {mode_val}")

ค่าว่าง 'เพศ' ค่าที่พบบ่อย: หญิง
ค่าว่าง 'คณะ' ค่าที่พบบ่อย: วิทยาศาสตร์


ข้อมูลหมวดหมู่: เพศ, คณะ เราจะใช้วิธี เติมด้วยค่าที่พบบ่อยที่สุด

In [15]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   รหัสนักศึกษา             19 non-null     int64  
 1   ชื่อ                     19 non-null     object 
 2   อายุ                     19 non-null     float64
 3   เพศ                      19 non-null     object 
 4   คณะ                      19 non-null     object 
 5   GPA                      19 non-null     float64
 6   ส่วนสูง_cm               19 non-null     float64
 7   น้ำหนัก_kg               19 non-null     float64
 8   ชั่วโมงเรียน_ต่อสัปดาห์  19 non-null     float64
 9   วันที่ลงทะเบียน          19 non-null     object 
dtypes: float64(5), int64(1), object(4)
memory usage: 1.6+ KB
None


จะแสดงว่าว่าไม่มีค่า (Non-Null) แล้ว

ขั้นตอนที่ 4: การสร้างคุณลักษณะใหม่ (Feature Engineering)

In [19]:
df['ส่วนสูง_m'] = df['ส่วนสูง_cm'] / 100
df['BMI'] = df['น้ำหนัก_kg'] / (df['ส่วนสูง_m'] ** 2)
df.drop('ส่วนสูง_m', axis=1, inplace=True)

In [20]:
print(df[['ชื่อ', 'ส่วนสูง_cm', 'น้ำหนัก_kg', 'BMI']].head())

       ชื่อ  ส่วนสูง_cm  น้ำหนัก_kg        BMI
0     สมชาย       170.0        65.0  22.491349
1    สมหญิง       165.0        58.0  21.303949
3  สมศักดิ์       168.0        70.0  24.801587
4      สมใจ       160.0        52.0  20.312500
5     กานต์       172.0        75.0  25.351541


การรวม Features: เราสามารถสร้าง BMI (ดัชนีมวลกาย) จากส่วนสูงและน้ำหนักได้

In [21]:
df_final = pd.get_dummies(df, columns=['เพศ', 'คณะ'], drop_first=True)

In [22]:
pd.set_option('display.max_columns', None)
print(df_final.head())

   รหัสนักศึกษา      ชื่อ    อายุ       GPA  ส่วนสูง_cm  น้ำหนัก_kg  \
0         64001     สมชาย  20.000  3.210000       170.0        65.0   
1         64002    สมหญิง  21.000  3.450000       165.0        58.0   
3         64004  สมศักดิ์  22.000  3.417333       168.0        70.0   
4         64005      สมใจ  19.000  3.890000       160.0        52.0   
5         64006     กานต์  20.875  3.120000       172.0        75.0   

   ชั่วโมงเรียน_ต่อสัปดาห์ วันที่ลงทะเบียน        BMI  เพศ_หญิง  \
0                     15.0      2024-02-13  22.491349     False   
1                     20.0      2023-12-31  21.303949      True   
3                     10.0      2024-01-31  24.801587     False   
4                     30.0      2024-02-04  20.312500      True   
5                     12.0      2024-01-28  25.351541     False   

   คณะ_วิศวกรรมศาสตร์  คณะ_ศิลปศาสตร์  
0               False           False  
1               False           False  
3               False           False  
4         

One-Hot Encoding: โมเดล Machine Learning ส่วนใหญ่ไม่สามารถทำงานกับข้อมูลที่เป็นข้อความได้โดยตรง เราต้องแปลงคอลัมน์ 'เพศ' และ 'คณะ' ให้เป็นตัวเลขก่อน วิธีที่นิยมคือ One-Hot Encoding

In [23]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [24]:
display(df_final)

,รหัสนักศึกษา,ชื่อ,อายุ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน,BMI,เพศ_หญิง,คณะ_วิศวกรรมศาสตร์,คณะ_ศิลปศาสตร์
0,64001,สมชาย,20.000,3.210000,170.000000,65.000000,15.000000,2024-02-13,22.491349,False,False,False
1,64002,สมหญิง,21.000,3.450000,165.000000,58.000000,20.000000,2023-12-31,21.303949,True,False,False
3,64004,สมศักดิ์,22.000,3.417333,168.000000,70.000000,10.000000,2024-01-31,24.801587,False,False,False
4,64005,สมใจ,19.000,3.890000,160.000000,52.000000,30.000000,2024-02-04,20.312500,True,True,False
5,64006,กานต์,20.875,3.120000,172.000000,75.000000,12.000000,2024-01-28,25.351541,False,True,False
6,64007,กิตติ,21.000,3.417333,158.000000,48.000000,35.000000,2024-02-08,19.227688,True,True,False
7,64008,จิรา,20.000,2.980000,180.000000,82.000000,18.000000,2024-02-05,25.308642,True,False,True
8,64009,ชนาธิป,23.000,3.340000,163.000000,55.000000,22.000000,2024-02-10,20.700817,False,False,True
9,64010,ดารา,21.000,3.560000,167.000000,60.000000,28.000000,2023-12-18,21.513859,True,False,True
10,64011,ธีระ,20.875,3.780000,167.277778,68.000000,21.176471,2024-01-21,24.301463,True,False,False
